In [91]:
from pydantic import BaseModel, Field, ValidationError, validator, field_validator
from typing import Annotated, Any, Dict, List, Optional, Set, Union
from pydantic.functional_validators import AfterValidator
import uuid
from abstractions.goap.entity import Entity, Attribute,RegistryHolder

name='Main Entrance' id='3ac4ae68-4172-4d93-bf5d-16483769a13a' open=Open(name='Open', id='87d1cf1b-b099-4700-b446-e7e0d6282a68', value=True) color=Color(name='Color', id='fbbfc195-006e-41d3-836d-7f17ecd7af19', value='white') material=Material(name='Material', id='3ae3c743-91ed-4f62-aab9-a979fb0ea5cb', value='wood') lock=Lock(name='Lock', id='0d27400a-a4c4-4faa-8576-c89b2951bf81', locked=Locked(name='Locked', id='061e91b1-9db2-4f33-abef-546ba7af604a', value=True))


In [94]:
# Specific attribute classes inheriting from Attribute
class Open(Attribute):
    value: bool = Field(default=True, description="Indicates if the entity is open or closed")

class Locked(Attribute):
    value: bool = Field(default=False, description="Indicates if the entity is locked or unlocked")

class Color(Attribute):
    value: str = Field(default="white", description="The color of the entity")

class Material(Attribute):
    value: str = Field(default="wood", description="The material of the entity")


# Door entity inheriting from Entity
class Door(Entity):
    open: Open = Field(default_factory=Open, description="Attribute to indicate door's open state")
    color: Color = Field(default_factory=Color, description="Attribute for the door's color")
    material: Material = Field(default_factory=Material, description="Attribute representing the door's material")

# Lock entity inheriting from Entity
class Lock(Entity):
    locked: Locked = Field(default_factory=Locked, description="Lock state, unlocked or locked")

# LockableDoor entity inheriting from Door and including a Lock entity
class LockableDoor(Door):
    lock: Lock = Field(default_factory=Lock, description="Lock entity for the door")

#alternatively, you can use the following to inherit from both Door and Lock instead of storing a lock Entity
class DoorLockable(Door,Lock):
    pass

# Example usage
print("Example usage with nested entities:")
try:
    lockable_door = LockableDoor(
        name="Main Entrance",
        open=Open(value=True),
        color=Color(value='white'),
        material=Material(value='wood'),
        lock=Lock(locked=Locked(value=True))
    )
    print(lockable_door)
except ValidationError as e:
    print(f"Validation error: {e}")
print("\nExample usage with multiple inheritance:")
try:
    door_lockable = DoorLockable(
        name="Main Entrance",
        open=Open(value=True),
        color=Color(value='white'),
        material=Material(value='wood'),
        locked=Locked(value=True)
    )
    print(door_lockable)
except ValidationError as e:
    print(f"Validation error: {e}")

Example usage with nested entities:
name='Main Entrance' id='bafe85ec-9de4-4a66-a0e4-1e71ed04e863' open=Open(name='Open', id='1d75e8ed-6ee4-4519-8d1f-53313301fc79', value=True) color=Color(name='Color', id='32312a5a-e4e3-42e1-8889-9785431d02de', value='white') material=Material(name='Material', id='f87cbc52-5fc0-4e93-b75f-75a257a5b4d3', value='wood') lock=Lock(name='Lock', id='e74038ba-f3f7-4ea3-a69e-94014f322845', locked=Locked(name='Locked', id='52d0dc97-31d5-4cea-9893-a56e3707e371', value=True))

Example usage with multiple inheritance:
name='Main Entrance' id='f080d353-bd7e-4875-a54e-a7b8b73b80ca' locked=Locked(name='Locked', id='00b67e7c-fff7-4581-89b9-714c2407afec', value=True) open=Open(name='Open', id='91b848da-13be-4441-8fd3-1908f3c9bea4', value=True) color=Color(name='Color', id='e9b20cd9-362b-4c6b-b1ee-0a24a8f0396d', value='white') material=Material(name='Material', id='e420e7c7-9382-495e-a443-65d0f4181d5b', value='wood')


In [89]:
lockable_door.all_attributes()

{'open': Open(name='Open', id='30231c83-2a80-4a93-95dc-8554a63db02d', value=True),
 'color': Color(name='Color', id='cd6af90f-c26a-40ee-9e11-b7f6e05604a3', value='white'),
 'material': Material(name='Material', id='2ce99ded-f72e-44ad-9ccb-83c962f111d9', value='wood'),
 'locked': Locked(name='Locked', id='faf9a0ae-3780-41d4-afcf-6de6fda0576a', value=True)}

In [76]:
Attribute.all_instances()

[Open(name='Open', id='c7c26627-f8e7-4d54-9212-51786c3992a6', value=True),
 Color(name='Color', id='9e6e483e-b1ee-41c6-8f7f-41c2ec9dfb90', value='white'),
 Material(name='Material', id='56d7d9db-8d9f-4ff0-a3e9-99cdef1ec534', value='wood'),
 Locked(name='Locked', id='8c6dd8ed-651d-461c-b893-b1ff831487ac', value=True)]

In [68]:
RegistryHolder.all_instances_by_type(Entity)

2

In [85]:
RegistryHolder.all_types()

['Locked', 'Material', 'Open', 'LockableDoor', 'Color', 'Lock']

In [95]:
def is_open(entity: Door) -> bool:
    if hasattr(entity, 'open') and isinstance(entity.open, Open):
        return entity.open.value
    elif not hasattr(entity, 'open'):
        raise ValueError("The provided entity does not have an 'open' attribute.")
    elif not isinstance(entity.open, Open):
        raise TypeError("The provided entity does not have an 'open' attribute of type Open.")

def is_locked(entity: LockableDoor) -> bool:
    if hasattr(entity, "lock") and hasattr(entity.lock, 'locked') and isinstance(entity.lock.locked, Locked):
        return entity.lock.locked.value
    elif not hasattr(entity, "lock"):
        raise ValueError("The provided entity does not have a 'lock' attribute.")
    elif not hasattr(entity.lock, 'locked'):
        raise ValueError("The provided entity.lock does not have a 'locked' attribute.")
    elif not isinstance(entity.lock.locked, Locked):
        raise TypeError("The provided entity.lock does not have a 'locked' attribute of type Locked.")
    
print(is_open(lockable_door))
print(is_locked(lockable_door))


True
True
